In [21]:
def to_hex_64chunks(t):
    res = []
    while t > 0:
        res.append(t % 2^64)
        t = t >> 64
    return res

def hex_64chunks_to_string(chunks):
    res = [hex(chunk).upper().replace('X', 'x') for chunk in chunks]
    res_string = '{' + ', '.join(res) + '}'
    return res_string

def from_hex_64chunks(chunks):
    res = 0
    for chunk in chunks[::-1]:
        res = res << 64
        res += chunk
    return res

def get_prime_details(pow_2, pow_3, cofactor, bits):
    res = {}

    p = 2^pow_2 * 3^pow_3 * cofactor - 1
    res["p"] = p
    res["p_bits"] = math.ceil(math.log2(p))

    R = 1 << bits
    res["R"] = R
    res["R_bits"] = bits

    p1 = int(1 + p)
    res["p1"] = p1
        
    px2 = p * 2
    res["px2"] = px2
    
    R = IntegerModRing(2^(bits))
    temp = R(p)
    temp = temp^-1
    temp = -1 * temp
    pinv_R = int(temp)
    res["pinv_R"] = pinv_R # Modulo AND additive inverse: p * pinv_R === -1 mod R

    P = IntegerModRing(p)
    R2 = 2^(bits * 2)
    temp = P(R2)
    R2_p = int(temp)
    res["R2_p"] = R2_p

    R = 2^(bits)
    temp = P(R)
    R_p = int(temp)
    res["R_p"] = R_p

    return res

def print_prime_details(prime_details):
    for key in prime_details.keys():
        if "bits" in key:
            print(key, ':', prime_details[key])
            continue
        print(key, ':', hex_64chunks_to_string(to_hex_64chunks(prime_details[key])))

def print_prime_details2(pow_2, pow_3, cofactor, bits):
    print('p = 2^{} * 3^{} * {} - 1'.format(pow_2, pow_3, cofactor))
    p = 2^pow_2 * 3^pow_3 * cofactor - 1
    
    print('p bits = {}'.format(math.log(p, 2)))

    print('prime')
    temp = p
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))

    print('primeplus1')
    temp = int(1 + p)
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))
        
    print('prime*2')
    temp = p * 2
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))
    
    print('Mont prime_inv mod R')
    R = IntegerModRing(2^(bits))
    temp = R(p)
    temp = temp^-1
    temp = -1 * temp
    temp = int(temp)
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))

    P = IntegerModRing(p)
    R2 = 2^(bits * 2)
    print('Mont R^2 mod prime')
    temp = P(R2)
    temp = int(temp)
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))

    print('Mont R mod prime')
    R = 2^(bits)
    temp = P(R)
    temp = int(temp)
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))

def check_supersingular(p):
    F2.<a> = GF(p^2, modulus=x^2 + 1) # Note; not using GF(p^2) because of a limitation in Sage
    print(F2)
    A = F2(6)
    B = F2(1)
    C = F2(1)
    S = B^1 * (1 - A^2/3)
    T = B^3 * A / 3 * (2*A^2/9 - 1)
    E = EllipticCurve(F2, [S, T]) # v^2 = u^3 + (B^2(1 - A^2/3))u + B^3 * A / 3(2A^2/9 - 1)
    print(E)
    return E.is_supersingular()

def to_mont(prime_details, elem):
    return (elem << prime_details["R_bits"]) % prime_details["p"]

def mont_red(prime_details, m_elem):
    m = ((m_elem & ((1 << prime_details["R_bits"]) - 1)) * prime_details["pinv_R"]) & ((1 << prime_details["R_bits"]) - 1)
    t = (m_elem + m * prime_details["p"]) >> prime_details["R_bits"]
    if t >= prime_details["p"]:
        print("@@@@@@@")
        return t - prime_details["p"]
    else:
        return t

def compute_y(E,x):
    R.<y> = PolynomialRing(E.base_ring())
    a = E.a_invariants()
    L = (y^2 + (a[0]*x + a[2])*y - (x^3 + a[1]*x^2 + a[3]*x + a[4])).roots()
    return [l[0] for l in L]

In [22]:
lA, lB = 2, 3
eA, eB = 216, 137 # p434 = 2^216*3^137-1
p = lA ^ eA * lB ^ eB - 1
assert p.is_prime()
assert p % 4 == 3 # Necessary for below curve to be supersingular.
#print("Is supersingular: ", check_supersingular(p))
prime_details = get_prime_details(eA, eB, 1, 448)
print_prime_details(prime_details)
assert(p == from_hex_64chunks(to_hex_64chunks(p)))

F2.<a> = GF(p^2, modulus=x^2 + 1)
A = F2(6)
B = F2(1)
C = F2(1)
S = B^1 * (1 - A^2/3)
T = B^3 * A / 3 * (2*A^2/9 - 1)
E = EllipticCurve(F2, [0, A, 0, 1, 0])
print(E)

p : {0xFFFFFFFFFFFFFFFF, 0xFFFFFFFFFFFFFFFF, 0xFFFFFFFFFFFFFFFF, 0xFDC1767AE2FFFFFF, 0x7BC65C783158AEA3, 0x6CFC5FD681C52056, 0x2341F27177344}
p_bits : 434
R : {0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x1}
R_bits : 448
p1 : {0x0, 0x0, 0x0, 0xFDC1767AE3000000, 0x7BC65C783158AEA3, 0x6CFC5FD681C52056, 0x2341F27177344}
px2 : {0xFFFFFFFFFFFFFFFE, 0xFFFFFFFFFFFFFFFF, 0xFFFFFFFFFFFFFFFF, 0xFB82ECF5C5FFFFFF, 0xF78CB8F062B15D47, 0xD9F8BFAD038A40AC, 0x4683E4E2EE688}
pinv_R : {0x1, 0x0, 0x0, 0xFDC1767AE3000000, 0x7BC65C783158AEA3, 0x6CFC5FD681C52056, 0x254B341F27177344}
R2_p : {0x28E55B65DCD69B30, 0xACEC7367768798C2, 0xAB27973F8311688D, 0x175CC6AF8D6C7C0B, 0xABCD92BF2DDE347E, 0x69E16A61C7686D9A, 0x25A89BCDD12A}
R_p : {0x742C, 0x0, 0x0, 0xB90FF404FC000000, 0xD801A4FB559FACD4, 0xE93254545F77410C, 0xECEEA7BD2EDA}
Elliptic Curve defined by y^2 = x^3 + 6*x^2 + x over Finite Field in a of size 24439423661345221551909145011457493619085780243761596511325807336205221239331976725970216671828618445898719026692884

In [23]:
m_xPA0 = from_hex_64chunks([0x05ADF455C5C345BF, 0x91935C5CC767AC2B, 0xAFE4E879951F0257, 0x70E792DC89FA27B1, 0xF797F526BB48C8CD, 0x2181DB6131AF621F, 0x00000A1C08B1ECC4])
m_xPA1 = from_hex_64chunks([0x74840EB87CDA7788, 0x2971AA0ECF9F9D0B, 0xCB5732BDF41715D5, 0x8CD8E51F7AACFFAA, 0xA7F424730D7E419F, 0xD671EB919A179E8C, 0x0000FFA26C5A924A])

m_xQA0 = from_hex_64chunks([0xFEC6E64588B7273B, 0xD2A626D74CBBF1C6, 0xF8F58F07A78098C7, 0xE23941F470841B03, 0x1B63EDA2045538DD, 0x735CFEB0FFD49215, 0x0001C4CB77542876])
m_xQA1 = from_hex_64chunks([0xADB0F733C17FFDD6, 0x6AFFBD037DA0A050, 0x680EC43DB144E02F, 0x1E2E5D5FF524E374, 0xE2DDA115260E2995, 0xA6E4B552E2EDE508, 0x00018ECCDDF4B53E])

m_xRA0 = from_hex_64chunks([0x01BA4DB518CD6C7D, 0x2CB0251FE3CC0611, 0x259B0C6949A9121B, 0x60E17AC16D2F82AD, 0x3AA41F1CE175D92D, 0x413FBE6A9B9BC4F3, 0x00022A81D8D55643])
m_xRA1 = from_hex_64chunks([0xB8ADBC70FC82E54A, 0xEF9CDDB0D5FADDED, 0x5820C734C80096A0, 0x7799994BAA96E0E4, 0x044961599E379AF8, 0xDB2B94FBF09F27E2, 0x0000B87FC716C0C6])

xPA0 = mont_red(prime_details, m_xPA0)
xPA1 = mont_red(prime_details, m_xPA1)
xPA = F2(xPA0 + xPA1 * a)
yPA_roots = compute_y(E, xPA)
#print(yPA_roots)

PA1 = E(xPA, yPA_roots[0])
PA2 = E(xPA, yPA_roots[1])

xQA0 = mont_red(prime_details, m_xQA0)
xQA1 = mont_red(prime_details, m_xQA1)
xQA = F2(xQA0 + xQA1 * a)
yQA_roots = compute_y(E, xQA)
#print(yQA_roots)

QA1 = E(xQA, yQA_roots[0])
QA2 = E(xQA, yQA_roots[1])

xRA0 = mont_red(prime_details, m_xRA0)
xRA1 = mont_red(prime_details, m_xRA1)
xRA = F2(xRA0 + xRA1 * a)

print("2^eA:\t\t", 2^eA)
print("Order PA:\t", PA1.order())
print("Order QA:\t", QA1.order())
print("Order PA + QA:\t", (PA1 + QA1).order())

print("xRA:\t\t", xRA)
print((PA1 - QA1)[0] == xRA)
print((PA2 - QA1)[0] == xRA)
print((PA1 - QA2)[0] == xRA)
print((PA2 - QA2)[0] == xRA)

2^eA:		 105312291668557186697918027683670432318895095400549111254310977536
Order PA:	 105312291668557186697918027683670432318895095400549111254310977536
Order QA:	 105312291668557186697918027683670432318895095400549111254310977536
Order PA + QA:	 105312291668557186697918027683670432318895095400549111254310977536
xRA:		 17623338845092751517427595119320347017334966146230013113905734683582704966390296970846105400364294574370981828797535336936167097772*a + 10548244869191429978994573331033429460802791853191679921432716242390096998215982561051229194803656270150791181542353263212179039510
True
False
False
True


In [24]:
m_xPB0 = from_hex_64chunks([0x6E5497556EDD48A3, 0x2A61B501546F1C05, 0xEB919446D049887D, 0x5864A4A69D450C4F, 0xB883F276A6490D2B, 0x22CC287022D5F5B9, 0x0001BED4772E551F])
m_xPB1 = from_hex_64chunks([0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000])

m_xQB0 = from_hex_64chunks([0xFAE2A3F93D8B6B8E, 0x494871F51700FE1C, 0xEF1A94228413C27C, 0x498FF4A4AF60BD62, 0xB00AD2A708267E8A, 0xF4328294E017837F, 0x000034080181D8AE])
m_xQB1 = from_hex_64chunks([0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000])

m_xRB0 = from_hex_64chunks([0x283B34FAFEFDC8E4, 0x9208F44977C3E647, 0x7DEAE962816F4E9A, 0x68A2BA8AA262EC9D, 0x8176F112EA43F45B, 0x02106D022634F504, 0x00007E8A50F02E37])
m_xRB1 = from_hex_64chunks([0xB378B7C1DA22CCB1, 0x6D089C99AD1D9230, 0xEBE15711813E2369, 0x2B35A68239D48A53, 0x445F6FD138407C93, 0xBEF93B29A3F6B54B, 0x000173FA910377D3])

xPB0 = mont_red(prime_details, m_xPB0)
xPB1 = mont_red(prime_details, m_xPB1)
xPB = F2(xPB0 + xPB1 * a)
yPB_roots = compute_y(E, xPB)
#print(yPB_roots)

PB1 = E(xPB, yPB_roots[0])
PB2 = E(xPB, yPB_roots[1])

xQB0 = mont_red(prime_details, m_xQB0)
xQB1 = mont_red(prime_details, m_xQB1)
xQB = F2(xQB0 + xQB1 * a)
yQB_roots = compute_y(E, xQB)
#print(yQB_roots)

QB1 = E(xQB, yQB_roots[0])
QB2 = E(xQB, yQB_roots[1])

xRB0 = mont_red(prime_details, m_xRB0)
xRB1 = mont_red(prime_details, m_xRB1)
xRB = F2(xRB0 + xRB1 * a)

print("3^eB:\t\t", 3^eB)
print("Order PB:\t", PB1.order())
print("Order QB:\t", QB1.order())
print("Order PB + QB:\t", (PB1 + QB1).order())

print("xRB:\t\t", xRB)
print((PB1 - QB1)[0] == xRB)
print((PB2 - QB1)[0] == xRB)
print((PB1 - QB2)[0] == xRB)
print((PB2 - QB2)[0] == xRB)

3^eB:		 232066203043628532565045340531182604896544238770765380550355483363
Order PB:	 232066203043628532565045340531182604896544238770765380550355483363
Order QB:	 232066203043628532565045340531182604896544238770765380550355483363
Order PB + QB:	 232066203043628532565045340531182604896544238770765380550355483363
xRB:		 14167827257511306746606440016400170231086622175754382579855491498256779752299521404090563911050166061448571907478184141518856743652*a + 19978714732817982296321998790126652405475699311529669091328949981490769847281914491438519436155586335833863255694913096932863948652
True
False
False
True


In [26]:
BITS = 64
log3_2 = math.log(2, 3)
for pow_2 in range(10, 32):
    for pow_3 in range(math.floor(log3_2 * pow_2) - 5, math.floor(log3_2 * pow_2) + 5):
        if is_prime(2^pow_2 * 3^pow_3 - 1):
            if 1/5 <= 2^pow_2 /  3^pow_3 <= 5/1:
                p = 2^pow_2 * 3^pow_3 - 1
                if check_supersingular(p):
                    print_prime_details(get_prime_details(pow_2, pow_3, 1, BITS))
                    print('')
#print_prime_details(32, 20, 23, 128)

Finite Field in a of size 17915903^2
Elliptic Curve defined by y^2 = x^3 + 17915892*x + 14 over Finite Field in a of size 17915903^2
p : {0x1115FFF}
p_bits : 25
R : {0x0, 0x1}
R_bits : 64
p1 : {0x1116000}
px2 : {0x222BFFE}
pinv_R : {0xB13A7D6DE5116001}
R2_p : {0x6B7757}
R_p : {0xE6C873}

Finite Field in a of size 214990847^2
Elliptic Curve defined by y^2 = x^3 + 214990836*x + 14 over Finite Field in a of size 214990847^2
p : {0xCD07FFF}
p_bits : 28
R : {0x0, 0x1}
R_bits : 64
p1 : {0xCD08000}
px2 : {0x19A0FFFE}
pinv_R : {0xA7C055D04CD08001}
R2_p : {0x2626A4D}
R_p : {0x7582CE3}

Finite Field in a of size 60183678025727^2
Elliptic Curve defined by y^2 = x^3 + 60183678025716*x + 14 over Finite Field in a of size 60183678025727^2
p : {0x36BC9ABFFFFF}
p_bits : 46
R : {0x0, 0x1}
R_bits : 64
p1 : {0x36BC9AC00000}
px2 : {0x6D79357FFFFE}
pinv_R : {0xA78BC6BC9AC00001}
R2_p : {0x25512E2FF61A}
R_p : {0x172AE9C4AD4B}

